# Tutorial on getting hands on the event data for a single match.
-----


*author: alihasankhan*

*twitter: @rockingAli5* 


# Import modules

In [1]:
# !git clone "https://github.com/Ali-Hasan-Khan/Scrape-Whoscored-Event-Data.git"

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import main
# import visuals
import seaborn as sns
import requests
# import utils
import os
%load_ext autoreload
%autoreload 2

/tmp/ipykernel_86452/264210428.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Get match data

If you get the following exception, try running the cell again:<br>
`NoSuchElementException: Message: unknown error: cannot determine loading status
from no such window`

In [112]:
driver = webdriver.Firefox()

In [5]:
match_url = "https://www.scoresway.com/en_GB/soccer/primera-divisi%C3%B3n-2024-2025/4x7uzww3jur4re7sgt3mslyj8/match/view/6y0rev1t8s1s8hppafgrnmuc4/match-stats"
driver.get(match_url)

In [51]:
tables = driver.find_element(By.XPATH, '//*[@id="Opta_1"]/div/div/div')
tables

<selenium.webdriver.remote.webelement.WebElement (session="f9a1d8c3-9f36-4005-9e81-96ef35686ead", element="ae3194dc-84f6-43b0-852c-673846a90b60")>

In [15]:
from bs4 import BeautifulSoup as soup
tables = soup(tables.get_attribute('innerHTML')).find_all('table')

In [ ]:
def getMatchStats(driver, url):
    import time
    driver.get(url)
    time.sleep(5) # wait for elemnt to load

    match_metadata_table = driver.find_element(By.XPATH, '//*[@id="Opta_0"]/div/div/table')
    match_metadata_table = soup(match_metadata_table.get_attribute('innerHTML'))
    home_team = match_metadata_table.findAll('td', class_='Opta-Team')[0].text.strip()
    home_score = match_metadata_table.findAll('td', class_='Opta-Score')[0].text.strip()
    away_team = match_metadata_table.findAll('td', class_='Opta-Team')[1].text.strip()
    away_score = match_metadata_table.findAll('td', class_='Opta-Score')[1].text.strip()

    # Extract competition and date
    competition = match_metadata_table.find('span', class_='Opta-Competition').text.strip()
    date = match_metadata_table.find('span', class_='Opta-Date').text.strip().split(' ')[0]
    time = match_metadata_table.find('span', class_='Opta-Date').text.strip().split(' ')[1]
    home_team, home_score, away_team, away_score, competition, date, time

    tables = driver.find_element(By.XPATH, '//*[@id="Opta_1"]/div/div/div') # //*[@id="Opta_1"]/div/div/div
    tables = soup(tables.get_attribute('innerHTML')).find_all('table')
    result = []
    result.append({"Match Metadata": {"Home Team": home_team, "Home Score": home_score, "Away Team": away_team, "Away Score": away_score, "Competition": competition, "Date": date, "Time": time}})
    for table in tables:
        rows = table.find_all('tr')
        for i in range(0, len(rows), 2):  # Headings and data are in pairs
            heading = rows[i].find('th').text.strip()
            data = rows[i + 1].find_all('td')
            home = data[0].text.strip()
            away = data[2].text.strip()
            result.append({heading:{"Home": home, "Away": away}})
    return result

match_stats_url = "https://www.scoresway.com/en_GB/soccer/premier-league-2024-2025/9n12waklv005j8r32sfjj2eqc/match/view/b9jqvdwj3ykf2ca6bhvx45iqc/match-stats"
res = getMatchStats(driver, match_stats_url)
print(res)

[{'Match Metadata': {'Home Team': 'Southampton FC', 'Home Score': '2', 'Away Team': 'Liverpool FC', 'Away Score': '3', 'Competition': 'Premier League', 'Date': '24/11/2024', 'Time': '19:30'}}, {'Possession': {'Home': '37.8%', 'Away': '62.2%'}}, {'Duels success rate': {'Home': '48.7%', 'Away': '51.3%'}}, {'Aerial duels won': {'Home': '57.1%', 'Away': '42.9%'}}, {'Interceptions': {'Home': '5', 'Away': '8'}}, {'Offsides': {'Home': '0', 'Away': '1'}}, {'Corners won': {'Home': '3', 'Away': '10'}}, {'Passes': {'Home': '365', 'Away': '600'}}, {'Long passes': {'Home': '29', 'Away': '39'}}, {'Passing accuracy': {'Home': '83.0%', 'Away': '88.5%'}}, {"Passing accuracy in opponents' half (%)": {'Home': '76.5%', 'Away': '82.0%'}}, {'Crosses': {'Home': '8', 'Away': '30'}}, {'Crossing accuracy': {'Home': '37.5%', 'Away': '30.0%'}}, {'Goals': {'Home': '2', 'Away': '3'}}, {'Shots': {'Home': '7', 'Away': '27'}}, {'Shots on target': {'Home': '5', 'Away': '11'}}, {'Blocked shots': {'Home': '0', 'Away': '9

In [145]:
def getPlayerStats(driver, player_stats_url):
    import time
    driver.get(player_stats_url)
    time.sleep(5) # wait for elemnt to load

    tables = driver.find_element(By.XPATH, '//*[@id="mts_home"]/div/div/div[1]')
    tables = soup(tables.get_attribute('innerHTML'))


    dfs = []

    # Find all the category sections (li elements containing player stats)
    categories = tables.select("ul.Opta-TabbedContent > li")

    for category in categories:
        # Extract the category name
        category_name = category.find("h3").text.strip()

        # Find the table for this category
        table = category.find("table", class_="Opta-Striped")
        if not table:
            continue

        # Extract table headers (only from <thead>)
        headers = [header.text.strip() for header in table.find("thead").find_all("th") if header.text.strip()]

        # Extract player rows
        rows = []
        for row in table.find("tbody").find_all("tr"):
            player_name = row.find("th").text.strip()  # Player name in the first <th> of the row
            stats = [cell.text.strip() or None for cell in row.find_all("td")]  # Other stats in <td>
            rows.append([player_name] + stats)

        # Create a DataFrame for this category
        df = pd.DataFrame(rows, columns=["Player"] + headers)

        # Add category-specific column prefixes (to distinguish overlapping column names)
        df = df.rename(columns=lambda x: f"{category_name}_{x}" if x != "Player" else x)
        dfs.append(df)

    # Merge all DataFrames on the Player column
    combined_df = dfs[0]
    for df in dfs[1:]:
        combined_df = pd.merge(combined_df, df, on="Player", how="outer")


    away_btn = driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[3]/div/div/ul/li[2]').click()
    time.sleep(5) # wait for elemnt to load

    tables = driver.find_element(By.XPATH, '//*[@id="mts_away"]/div/div/div[1]') 
    tables = soup(tables.get_attribute('innerHTML'))
    away_dfs = []

    # Repeat the same parsing logic as for home players to extract away players' data
    categories = tables.select("ul.Opta-TabbedContent > li")
    for category in categories:
        # Extract the category name
        category_name = category.find("h3").text.strip()

        # Find the table for this category
        table = category.find("table", class_="Opta-Striped")
        if not table:
            continue

        # Extract table headers (only from <thead>)
        headers = [header.text.strip() for header in table.find("thead").find_all("th") if header.text.strip()]

        # Extract player rows
        rows = []
        for row in table.find("tbody").find_all("tr"):
            player_name = row.find("th").text.strip()  # Player name in the first <th> of the row
            stats = [cell.text.strip() or None for cell in row.find_all("td")]  # Other stats in <td>
            rows.append([player_name] + stats)

        # Create a DataFrame for this category
        df = pd.DataFrame(rows, columns=["Player"] + headers)

        # Add category-specific column prefixes (to distinguish overlapping column names)
        df = df.rename(columns=lambda x: f"{category_name}_{x}" if x != "Player" else x)
        away_dfs.append(df)

    # Merge all away DataFrames on the Player column
    away_combined_df = away_dfs[0]
    for df in away_dfs[1:]:
        away_combined_df = pd.merge(away_combined_df, df, on="Player", how="outer")

    # Add 'team' column to both home and away DataFrames
    combined_df["team"] = "home"
    away_combined_df["team"] = "away"

    # Concatenate home and away data
    final_df = pd.concat([combined_df, away_combined_df], ignore_index=True)

    return final_df

player_stats_url = "https://www.scoresway.com/en_GB/soccer/primera-divisi%C3%B3n-2024-2025/4x7uzww3jur4re7sgt3mslyj8/match/view/6y0rev1t8s1s8hppafgrnmuc4/player-stats"
df = getPlayerStats(driver, player_stats_url)
df.head()

,Player,General_MP,General_T,General_D,General_D%,General_AD,General_ADW,General_SD,General_FW,General_O,...,Goalkeeping_SAV,Goalkeeping_SFonT,Goalkeeping_SS (%),Goalkeeping_KS,Goalkeeping_KS (%),Goalkeeping_CC,Goalkeeping_CnC,Goalkeeping_D,Goalkeeping_Pu,team
0,Alonso,90,59,2,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,home
1,Aspas,89,46,8,75%,2,100%,None,2,1,...,None,None,None,None,None,None,None,None,None,home
2,Bamba,62,29,5,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,home
3,Beltrán,75,37,4,50%,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,home
4,Douvikas,75,15,5,40%,1,None,None,1,1,...,None,None,None,None,None,None,None,None,None,home
